In [ ]:
!nvidia-smi

Mon Aug 28 07:20:30 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Finetune Llama-2-7b on a Google colab
Welcome to this Google Colab notebook that shows how to fine-tune the recent Llama-2-7b model on a single Google colab and turn it into a chatbot

We will leverage PEFT library from Hugging Face ecosystem, as well as QLoRA for more memory efficient finetuning

# Setup
Run the cells below to setup and install the required libraries. For our experiment we will need `accelerate`, `peft`, `transformers`, `datasets` and `TRL` to leverage the recent `SFTTrainer`. We will use `bitsandbytes` to [quantize the base model into 4bit](https://huggingface.co/blog/4bit-transformers-bitsandbytes). We will also install einops as it is a requirement to load Falcon models.

In [ ]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


# Dataset

In [ ]:
from huggingface_hub import login
login()

In [ ]:
from datasets import load_dataset

dataset_name = 'nampdn-ai/tiny-textbooks'
dataset = load_dataset(dataset_name, split="train")

Resolving data files:   0%|          | 0/42 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset[1]

{'text': 'Have you seen the “flag this blog” button now being published in the top bar of BlogSpot.com blogs? Interesting.Blogger explains this tool as a way to:. the “Flag?” button allows the blogging community to easily note questionable content, which in turn helps us take action when needed. So we’re relying on you, the users, to be our eyes on the web, and to let us know of potential issues that are important to you.. I have other thoughts on it. I see it as a way for Blogger to protect itself - using the strength of the community of users - from sickos posting about criminal acts or tendencies on their tool. Not sure what I’m talking about? See this for background.',
 'source': 'refinedweb',
 's': 5,
 'len': 681,
 'idx': 166349,
 'textbook': '**Lesson: How to Use the "Flag This Blog" Feature on BlogSpot**\n\n**Introduction:**\n\nThe "Flag This Blog" feature on BlogSpot is a tool that allows users to report blogs that contain questionable content. This feature is designed to help 

In [ ]:
print(dataset)

Dataset({
    features: ['text', 'source', 's', 'len', 'idx', 'textbook'],
    num_rows: 420000
})


In [ ]:
from datasets import Dataset

# Assuming `dataset` is your Dataset object
dataset = dataset.map(lambda example: {'text': example['prompt'] + example['output']})

Map:   0%|          | 0/420000 [00:00<?, ? examples/s]

KeyError: ignored

In [ ]:
dataset

Dataset({
    features: ['text', 'source', 's', 'len', 'idx', 'textbook'],
    num_rows: 420000
})

# Loading the Model

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "TinyPixel/Llama-2-7B-bf16-sharded"

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
from peft import LoraConfig, get_peft_model

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

In [ ]:
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

NameError: ignored

In [ ]:
from transformers import TrainingArguments

In [ ]:
output_dir = "./results"
per_device_train_batch_size = 1
gradient_accumulation_steps = 2
optim = "paged_adamw_32bit"
save_steps = 1
num_train_epochs = 4
logging_steps = 1
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 20
warmup_ratio = 0.03
lr_scheduler_type = "linear"

In [ ]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [ ]:
from trl import SFTTrainer

In [ ]:
max_seq_length = 2048

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:107: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/420000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:207: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [ ]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [ ]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,1.568100
2,1.909800
3,1.694300
4,1.808200
5,1.875300
6,1.751100
7,1.867800
8,1.384300
9,1.766200
10,2.293200


TrainOutput(global_step=20, training_loss=1.958068734407425, metrics={'train_runtime': 290.9018, 'train_samples_per_second': 0.138, 'train_steps_per_second': 0.069, 'total_flos': 833804595314688.0, 'train_loss': 1.958068734407425, 'epoch': 0.0})

In [ ]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained("outputs")

In [ ]:
lora_config = LoraConfig.from_pretrained('outputs')
model = get_peft_model(model, lora_config)

In [ ]:
model.push_to_hub("oozyii/Llama2_ft_tinytext",create_pr=1)

adapter_model.bin:   0%|          | 0.00/134M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/oozyii/Llama2_ft_tinytext/commit/9d20c3f47c2373af1055c9f49dd8b2d0cdc6e7d7', commit_message='Upload model', commit_description='', oid='9d20c3f47c2373af1055c9f49dd8b2d0cdc6e7d7', pr_url='https://huggingface.co/oozyii/Llama2_ft_tinytext/discussions/1', pr_revision='refs/pr/1', pr_num=1)

In [ ]:
dataset['text'][0]

'Dariusz Nosorowski. Managing Director, Atol Poland. Dariusz - a salesman, manager, lawyer with MBA title, SGH Warsaw School of Economics graduate and accomplished sales manager. He has nearly 20 years of sales experience at various organisational levels and both - large-scale international organisations as well as local startups. Leader of change (restructuring, market segmentation), effective creator and implementer of the selected strategies. He implemented recovery plans in the weakest departments, which later ranked among the best in the company. A coach and mentor, effectively builds successful sales teams, motivates employees to achieve high performance. He willingly shares his knowledge and experience in the work sphere collaborating with companies as an active salesman and consultant.'

In [ ]:
dataset['output'][0]

KeyError: ignored

In [ ]:
text = dataset['text'][0]
device = "cuda:0"

In [ ]:
inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1421: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


Dariusz Nosorowski. Managing Director, Atol Poland. Dariusz - a salesman, manager, lawyer with MBA title, SGH Warsaw School of Economics graduate and accomplished sales manager. He has nearly 20 years of sales experience at various organisational levels and both - large-scale international organisations as well as local startups. Leader of change (restructuring, market segmentation), effective creator and implementer of the selected strategies. He implemented recovery plans in the weakest departments, which later ranked among the best in the company. A coach and mentor, effectively builds successful sales teams, motivates employees to achieve high performance. He willingly shares his knowledge and experience in the work sphere collaborating with companies as an active salesman and consultant.
Dariusz Nosorowski
Managing Director, Atol Poland
Dariusz - a salesman, manager, lawyer with MBA title, SGH Warsaw School of Economics graduate and accomplished sales manager. He
